# Interactive Visualization with Folium

Interactive visualizations allow users to interact with data on maps, providing a more dynamic and engaging way to explore geographic information. In this notebook, we'll use the `folium` library to create interactive maps for visualizing SpaceX Falcon 9 launch sites and their success rates.

---

# Step 1: Installing Required Libraries

We'll start by installing the `folium` and `pandas` libraries. `Folium` is used for creating interactive maps, while `pandas` is used for data manipulation.


In [3]:
!pip install folium pandas
pip install pandas

Defaulting to user installation because normal site-packages is not writeable


---

# Step 2: Importing Libraries

We import the necessary libraries including `folium` for mapping, `pandas` for data handling, and `folium.plugins` for additional map functionalities.


In [5]:
import folium
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

---

# Step 3: Loading the Dataset

We'll load the SpaceX Falcon 9 launch data from a CSV file. This dataset contains information about launch sites and their geographic coordinates.


In [9]:
# Load the dataset
spacex_df = pd.read_csv("C:\\personal\\spacex-ibm\\dataset_part_2.csv")

---

# Step 4: Creating a Base Map with Launch Sites

We create a base map centered around the mean coordinates of the launch sites. This map will be used to add markers for each launch site.


In [10]:
# Extract the relevant columns for launch sites
launch_sites_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()
site_map = folium.Map(location=[launch_sites_df['Latitude'].mean(), launch_sites_df['Longitude'].mean()], zoom_start=5)

---

# Step 5: Adding Markers to the Map

We use `MarkerCluster` to group markers for different launch sites and add them to the map. Each marker represents a launch site, and clicking on it shows the site name.


In [12]:
# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers to the cluster
for index, site in launch_sites_df.iterrows():
    folium.Marker(
        location=[site['Latitude'], site['Longitude']],
        popup=site['LaunchSite'],
        icon=DivIcon(icon_size=(150,36), icon_anchor=(0,0),
                     html=f'<div style="font-size: 12pt">{site["LaunchSite"]}</div>')
    ).add_to(marker_cluster)


---

# Step 6: Adding Mouse Position Plugin

We add a `MousePosition` plugin to display the latitude and longitude of the cursor location on the map.


In [13]:
from folium.plugins import MousePosition

MousePosition().add_to(site_map)


In [14]:
# Display the map with the launch sites marked
site_map


---

# Step 7: Loading Additional Data from a URL

We'll download a new dataset from an external URL and load it into a DataFrame for additional analysis.


In [18]:
import requests
import pandas as pd
import io
# URL of the CSV file
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Load the CSV content into a pandas DataFrame
    spacex_df = pd.read_csv(io.StringIO(response.text))
    print("Data downloaded and loaded into DataFrame successfully.")
else:
    print(f"Failed to download data. Status code: {response.status_code}")

# Display the first few rows of the DataFrame
spacex_df.head()


Data downloaded and loaded into DataFrame successfully.


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [19]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


---

# Step 8: Creating a Map Focused on NASA Johnson Space Center

We create a map centered around NASA Johnson Space Center and add a circle and marker to indicate its location.


In [20]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [21]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

---

# Step 9: Adding Launch Sites to the Map

We add markers and circles for each launch site, using color coding to indicate the launch site name and adding a popup for additional details.


In [23]:
for index, row in launch_sites_df.iterrows():
    launch_site = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Add a circle for each launch site
    circle = folium.Circle(
        launch_site,
        radius=1000,
        color='#blue',
        fill=True
    ).add_child(folium.Popup(site_name))
    
    # Add a marker for each launch site
    marker = folium.Marker(
        launch_site,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#blue;"><b>%s</b></div>' % site_name,
        )
    )
    
    # Add to the map
    site_map.add_child(circle)
    site_map.add_child(marker)


In [24]:
site_map

---

# Step 10: Interactive Map with Launch Success Rates

We enhance the map by adding markers with color coding based on the success or failure of launches. Each marker is added to a `MarkerCluster` to improve map readability.


In [25]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through the DataFrame and add markers for each launch
for index, row in spacex_df.iterrows():
    launch_site = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    launch_class = row['class']
    
    # Define color based on launch outcome
    marker_color = 'green' if launch_class == 1 else 'red'
    
    # Add a marker for each launch site
    marker = folium.Marker(
        launch_site,
        icon=folium.Icon(color=marker_color, icon='rocket'),
        popup=f"{site_name} - {'Success' if launch_class == 1 else 'Failure'}"
    )
    
    # Add the marker to the marker cluster
    marker.add_to(marker_cluster)

# Display the map
site_map

In [27]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [28]:
marker_cluster = MarkerCluster()
